In [1]:
import pandas as pd
import dask.dataframe as dd
import tqdm
import numpy as np
import datetime as dt

In [2]:
sc = pd.read_csv('./supply_chain/CustomerSupplier_Detail_Link.csv')
quarter = pd.DataFrame(
    {'quarter': [1, 2, 3, 4],
     'lnk': [1, 1, 1, 1]}
)
sc['lnk'] = 1
sc = sc.merge(quarter,
              how='left',
              on='lnk')
del quarter
del sc['lnk']

In [3]:
pol_covid = pd.read_stata('./risk_exposure/08_all_scores/pol_covid.dta')
cc = pd.read_csv('./risk_exposure/08_all_scores/cc_scores.csv')

In [4]:
supplier = sc[['GVKEY2', 'GVKEY1', 'y', 'quarter', 'SALECS']]
supplier.columns = ['firm_gvkey', 'supplier_gvkey', 'year', 'quarter', 'sale_cs']

customer = sc[['GVKEY1', 'GVKEY2', 'y', 'quarter', 'SALECS']]
customer.columns = ['firm_gvkey', 'customer_gvkey', 'year', 'quarter', 'sale_cs']

supplier = supplier[supplier['firm_gvkey'].notna() & supplier['supplier_gvkey'].notna()]
customer = customer[customer['firm_gvkey'].notna() & customer['customer_gvkey'].notna()]

In [5]:
pol_covid['date'] = pd.to_datetime(pol_covid['date'])
pol_covid['year'] = pol_covid.date.dt.year
pol_covid['month'] = pol_covid.date.dt.month
pol_covid['quarter'] = pd.np.select([(pol_covid['month'] <= 3) & (pol_covid['month'] >= 1),
                                     (pol_covid['month'] <= 6) & (pol_covid['month'] >= 4),
                                     (pol_covid['month'] <= 9) & (pol_covid['month'] >= 7),
                                     (pol_covid['month'] <= 12) & (pol_covid['month'] >= 10)],
                                    [1, 2, 3, 4])
pol_covid = pol_covid[['gvkey', 'year', 'quarter', 'Risk', 'Covid_Exposure']]
cc = cc[['gvkey', 'year', 'quarter', 'cc_expo_ew']]

C:\Users\Silas Liew\AppData\Local\Temp\ipykernel_29072\3741961639.py:4: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  pol_covid['quarter'] = pd.np.select([(pol_covid['month'] <= 3) & (pol_covid['month'] >= 1),


In [6]:
pol_agg = pd.DataFrame()
covid_agg = pd.DataFrame()
cc_agg = pd.DataFrame()
pol_agg['pol'] = pol_covid.groupby(['gvkey', 'year', 'quarter'])['Risk'].mean()
covid_agg['covid'] = pol_covid.groupby(['gvkey', 'year', 'quarter'])['Covid_Exposure'].mean()
cc_agg['cc'] = cc.groupby(['gvkey', 'year', 'quarter'])['cc_expo_ew'].mean()
pol_agg = pol_agg.reset_index()
covid_agg = covid_agg.reset_index()
cc_agg = cc_agg.reset_index()
pol_agg['gvkey'] = pol_agg['gvkey'].astype(float)
covid_agg['gvkey'] = covid_agg['gvkey'].astype(float)
cc_agg['gvkey'] = cc_agg['gvkey'].astype(float)

In [7]:
supplier = supplier.merge(pol_agg,
                          how='left',
                          left_on=['firm_gvkey', 'year', 'quarter'],
                          right_on=['gvkey', 'year', 'quarter'])
supplier = supplier.drop(['gvkey'], axis=1)
supplier = supplier.merge(covid_agg,
                          how='left',
                          left_on=['firm_gvkey', 'year', 'quarter'],
                          right_on=['gvkey', 'year', 'quarter'])
supplier = supplier.drop(['gvkey'], axis=1)
supplier = supplier.merge(cc_agg,
                          how='left',
                          left_on=['firm_gvkey', 'year', 'quarter'],
                          right_on=['gvkey', 'year', 'quarter'])
supplier = supplier.drop(['gvkey'], axis=1)
supplier = supplier.rename(columns={'pol': 'firm_pol',
                                    'covid': 'firm_covid',
                                    'cc': 'firm_cc'})


supplier = supplier.merge(pol_agg,
                          how='left',
                          left_on=['supplier_gvkey', 'year', 'quarter'],
                          right_on=['gvkey', 'year', 'quarter'])
supplier = supplier.drop(['gvkey'], axis=1)
supplier = supplier.merge(covid_agg,
                          how='left',
                          left_on=['supplier_gvkey', 'year', 'quarter'],
                          right_on=['gvkey', 'year', 'quarter'])
supplier = supplier.drop(['gvkey'], axis=1)
supplier = supplier.merge(cc_agg,
                          how='left',
                          left_on=['supplier_gvkey', 'year', 'quarter'],
                          right_on=['gvkey', 'year', 'quarter'])
supplier = supplier.drop(['gvkey'], axis=1)
supplier = supplier.rename(columns={'pol': 'supplier_pol',
                                    'covid': 'supplier_covid',
                                    'cc': 'supplier_cc'})


supplier = supplier.drop_duplicates()
supplier = supplier[supplier['year'] >= 2006]

In [8]:
customer = customer.merge(pol_agg,
                          how='left',
                          left_on=['firm_gvkey', 'year', 'quarter'],
                          right_on=['gvkey', 'year', 'quarter'])
customer = customer.drop(['gvkey'], axis=1)
customer = customer.merge(covid_agg,
                          how='left',
                          left_on=['firm_gvkey', 'year', 'quarter'],
                          right_on=['gvkey', 'year', 'quarter'])
customer = customer.drop(['gvkey'], axis=1)
customer = customer.merge(cc_agg,
                          how='left',
                          left_on=['firm_gvkey', 'year', 'quarter'],
                          right_on=['gvkey', 'year', 'quarter'])
customer = customer.drop(['gvkey'], axis=1)
customer = customer.rename(columns={'pol': 'firm_pol',
                                    'covid': 'firm_covid',
                                    'cc': 'firm_cc'})


customer = customer.merge(pol_agg,
                          how='left',
                          left_on=['customer_gvkey', 'year', 'quarter'],
                          right_on=['gvkey', 'year', 'quarter'])
customer = customer.drop(['gvkey'], axis=1)
customer = customer.merge(covid_agg,
                          how='left',
                          left_on=['customer_gvkey', 'year', 'quarter'],
                          right_on=['gvkey', 'year', 'quarter'])
customer = customer.drop(['gvkey'], axis=1)
customer = customer.merge(cc_agg,
                          how='left',
                          left_on=['customer_gvkey', 'year', 'quarter'],
                          right_on=['gvkey', 'year', 'quarter'])
customer = customer.drop(['gvkey'], axis=1)
customer = customer.rename(columns={'pol': 'customer_pol',
                                    'covid': 'customer_covid',
                                    'cc': 'customer_cc'})


customer = customer.drop_duplicates()
customer = customer[customer['year'] >= 2006]

In [9]:
sc = pd.read_csv('./supply_chain/CustomerSupplier_Detail_Link.csv')
sc['SALECS'] = np.where(sc['SALECS'] < 0, 0, sc['SALECS'])
customer_sale_t = sc.groupby(['GVKEY1', 'y'])['SALECS'].sum().reset_index(0).rename(columns={'SALECS': 'sale_t'}).reset_index(0)
supplier_sale_t = sc.groupby(['GVKEY2', 'y'])['SALECS'].sum().reset_index(0).rename(columns={'SALECS': 'sale_t'}).reset_index(0)
supplier = supplier.merge(supplier_sale_t, how='left', left_on=['firm_gvkey', 'year'], right_on=['GVKEY2', 'y'])
customer = customer.merge(customer_sale_t, how='left', left_on=['firm_gvkey', 'year'], right_on=['GVKEY1', 'y'])
supplier['sale_r'] = supplier['sale_cs'] / supplier['sale_t']
customer['sale_r'] = supplier['sale_cs'] / supplier['sale_t']
supplier = supplier.drop(['GVKEY2', 'y'], axis=1)
customer = customer.drop(['GVKEY1', 'y'], axis=1)

In [10]:
supplier['supplier_pol_weighted'] = supplier['supplier_pol'] * supplier['sale_r']
supplier['supplier_covid_weighted'] = supplier['supplier_covid'] * supplier['sale_r']
supplier['supplier_cc_weighted'] = supplier['supplier_cc'] * supplier['sale_r']
customer['customer_pol_weighted'] = customer['customer_pol'] * customer['sale_r']
customer['customer_covid_weighted'] = customer['customer_covid'] * customer['sale_r']
customer['customer_cc_weighted'] = customer['customer_cc'] * customer['sale_r']

In [13]:
supplier = supplier.dropna(subset=['firm_pol', 'firm_covid', 'firm_cc'])
customer = customer.dropna(subset=['firm_pol', 'firm_covid', 'firm_cc'])
supplier['larger_supplier_pol'] = np.where(supplier['supplier_pol'] > supplier['firm_pol'], 1, 0)
supplier['larger_supplier_covid'] = np.where(supplier['supplier_covid'] > supplier['firm_covid'], 1, 0)
supplier['larger_supplier_cc'] = np.where(supplier['supplier_cc'] > supplier['firm_cc'], 1, 0)
customer['larger_customer_pol'] = np.where(customer['customer_pol'] > customer['firm_pol'], 1, 0)
customer['larger_customer_covid'] = np.where(customer['customer_covid'] > customer['firm_covid'], 1, 0)
customer['larger_customer_cc'] = np.where(customer['customer_cc'] > customer['firm_cc'], 1, 0)

In [ ]:
larger_supplier_pol_prop = pd.DataFrame()
larger_supplier_pol_prop['prop'] = supplier.groupby(['year', 'quarter']).apply(lambda x: x.loc[x['larger_supplier_pol'] == 1, 'firm_gvkey'].nunique() / x['firm_gvkey'].nunique())
larger_supplier_pol_prop['firm_count'] = supplier.groupby(['year', 'quarter'])['firm_gvkey'].nunique()
larger_supplier_covid_prop = pd.DataFrame()
larger_supplier_covid_prop['prop'] = supplier.groupby(['year', 'quarter']).apply(lambda x: x.loc[x['larger_supplier_covid'] == 1, 'firm_gvkey'].nunique() / x['firm_gvkey'].nunique())
larger_supplier_covid_prop['firm_count'] = supplier.groupby(['year', 'quarter'])['firm_gvkey'].nunique()
larger_supplier_cc_prop = pd.DataFrame()
larger_supplier_cc_prop['prop'] = supplier.groupby(['year', 'quarter']).apply(lambda x: x.loc[x['larger_supplier_cc'] == 1, 'firm_gvkey'].nunique() / x['firm_gvkey'].nunique())
larger_supplier_cc_prop['firm_count'] = supplier.groupby(['year', 'quarter'])['firm_gvkey'].nunique()
larger_customer_pol_prop = pd.DataFrame()
larger_customer_pol_prop['prop'] = customer.groupby(['year', 'quarter']).apply(lambda x: x.loc[x['larger_customer_pol'] == 1, 'firm_gvkey'].nunique() / x['firm_gvkey'].nunique())
larger_customer_pol_prop['firm_count'] = customer.groupby(['year', 'quarter'])['firm_gvkey'].nunique()
larger_customer_covid_prop = pd.DataFrame()
larger_customer_covid_prop['prop'] = customer.groupby(['year', 'quarter']).apply(lambda x: x.loc[x['larger_customer_covid'] == 1, 'firm_gvkey'].nunique() / x['firm_gvkey'].nunique())
larger_customer_covid_prop['firm_count'] = customer.groupby(['year', 'quarter'])['firm_gvkey'].nunique()
larger_customer_cc_prop = pd.DataFrame()
larger_customer_cc_prop['prop'] = customer.groupby(['year', 'quarter']).apply(lambda x: x.loc[x['larger_customer_cc'] == 1, 'firm_gvkey'].nunique() / x['firm_gvkey'].nunique())
larger_customer_cc_prop['firm_count'] = customer.groupby(['year', 'quarter'])['firm_gvkey'].nunique()

In [33]:
supplier['larger_supplier_pol_w'] = np.where(supplier['supplier_pol_weighted'] > supplier['firm_pol'], 1, 0)
supplier['larger_supplier_covid_w'] = np.where(supplier['supplier_covid_weighted'] > supplier['firm_covid'], 1, 0)
supplier['larger_supplier_cc_w'] = np.where(supplier['supplier_cc_weighted'] > supplier['firm_cc'], 1, 0)
customer['larger_customer_pol_w'] = np.where(customer['customer_pol_weighted'] > customer['firm_pol'], 1, 0)
customer['larger_customer_covid_w'] = np.where(customer['customer_covid_weighted'] > customer['firm_covid'], 1, 0)
customer['larger_customer_cc_w'] = np.where(customer['customer_cc_weighted'] > customer['firm_cc'], 1, 0)

In [34]:
larger_supplier_pol_prop_w = pd.DataFrame()
larger_supplier_pol_prop_w['prop'] = supplier.groupby(['year', 'quarter']).apply(lambda x: x.loc[x['larger_supplier_pol_w'] == 1, 'firm_gvkey'].nunique() / x['firm_gvkey'].nunique())
larger_supplier_pol_prop_w['firm_count'] = supplier.groupby(['year', 'quarter'])['firm_gvkey'].nunique()
larger_supplier_covid_prop_w = pd.DataFrame()
larger_supplier_covid_prop_w['prop'] = supplier.groupby(['year', 'quarter']).apply(lambda x: x.loc[x['larger_supplier_covid_w'] == 1, 'firm_gvkey'].nunique() / x['firm_gvkey'].nunique())
larger_supplier_covid_prop_w['firm_count'] = supplier.groupby(['year', 'quarter'])['firm_gvkey'].nunique()
larger_supplier_cc_prop_w = pd.DataFrame()
larger_supplier_cc_prop_w['prop'] = supplier.groupby(['year', 'quarter']).apply(lambda x: x.loc[x['larger_supplier_cc_w'] == 1, 'firm_gvkey'].nunique() / x['firm_gvkey'].nunique())
larger_supplier_cc_prop_w['firm_count'] = supplier.groupby(['year', 'quarter'])['firm_gvkey'].nunique()
larger_customer_pol_prop_w = pd.DataFrame()
larger_customer_pol_prop_w['prop'] = customer.groupby(['year', 'quarter']).apply(lambda x: x.loc[x['larger_customer_pol_w'] == 1, 'firm_gvkey'].nunique() / x['firm_gvkey'].nunique())
larger_customer_pol_prop_w['firm_count'] = customer.groupby(['year', 'quarter'])['firm_gvkey'].nunique()
larger_customer_covid_prop_w = pd.DataFrame()
larger_customer_covid_prop_w['prop'] = customer.groupby(['year', 'quarter']).apply(lambda x: x.loc[x['larger_customer_covid_w'] == 1, 'firm_gvkey'].nunique() / x['firm_gvkey'].nunique())
larger_customer_covid_prop_w['firm_count'] = customer.groupby(['year', 'quarter'])['firm_gvkey'].nunique()
larger_customer_cc_prop_w = pd.DataFrame()
larger_customer_cc_prop_w['prop'] = customer.groupby(['year', 'quarter']).apply(lambda x: x.loc[x['larger_customer_cc_w'] == 1, 'firm_gvkey'].nunique() / x['firm_gvkey'].nunique())
larger_customer_cc_prop_w['firm_count'] = customer.groupby(['year', 'quarter'])['firm_gvkey'].nunique()

In [38]:
wr = pd.ExcelWriter('./transcripts/scores/props_w.xlsx')
larger_customer_pol_prop_w.to_excel(wr, sheet_name='larger_customer_pol_prop_w')
larger_customer_covid_prop_w.to_excel(wr, sheet_name='larger_customer_covid_prop_w')
larger_customer_cc_prop_w.to_excel(wr, sheet_name='larger_customer_cc_prop_w')
larger_supplier_pol_prop_w.to_excel(wr, sheet_name='larger_supplier_pol_prop_w')
larger_supplier_covid_prop_w.to_excel(wr, sheet_name='larger_supplier_covid_prop_w')
larger_supplier_cc_prop_w.to_excel(wr, sheet_name='larger_supplier_cc_prop_w')
wr.save()

C:\Users\Silas Liew\AppData\Local\Temp\ipykernel_29072\550151989.py:8: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  wr.save()


In [15]:
firm = pd.concat([supplier[['firm_gvkey', 'year', 'quarter', 'firm_pol', 'firm_covid', 'firm_cc']], customer[['firm_gvkey', 'year', 'quarter', 'firm_pol', 'firm_covid', 'firm_cc']]]).drop_duplicates()
firm = firm.merge(supplier[['firm_gvkey', 'year', 'quarter', 'supplier_gvkey', 'supplier_pol_weighted', 'supplier_covid_weighted', 'supplier_cc_weighted']],
                  how='left',
                  on=['firm_gvkey', 'year', 'quarter'])
firm = firm.merge(customer[['firm_gvkey', 'year', 'quarter', 'customer_gvkey', 'customer_pol_weighted', 'customer_covid_weighted', 'customer_cc_weighted']],
                  how='left',
                  on=['firm_gvkey', 'year', 'quarter'])
# firm = firm.fillna(0)
firm['sc_pol'] = firm['firm_pol'] + firm['supplier_pol_weighted'] + firm['customer_pol_weighted']
firm['sc_covid'] = firm['firm_covid'] + firm['supplier_covid_weighted'] + firm['customer_covid_weighted']
firm['sc_cc'] = firm['firm_cc'] + firm['supplier_cc_weighted'] + firm['customer_cc_weighted']
firm = firm.drop_duplicates()

In [16]:
supplier.to_csv('./transcripts/scores/supplier.csv', index=False)
customer.to_csv('./transcripts/scores/customer.csv', index=False)
firm.to_csv('./transcripts/scores/firm.csv', index=False)

In [2]:
supplier = pd.read_csv('./transcripts/scores/supplier.csv')
customer = pd.read_csv('./transcripts/scores/customer.csv')

In [10]:
supplier = supplier.dropna()
equal_weighted_first_degree_supplier_score = pd.DataFrame()
equal_weighted_first_degree_supplier_score['ewfdsc_pol'] = supplier.groupby(['firm_gvkey', 'year', 'quarter'])['supplier_pol'].mean()
equal_weighted_first_degree_supplier_score['ewfdsc_covid'] = supplier.groupby(['firm_gvkey', 'year', 'quarter'])['supplier_covid'].mean()
equal_weighted_first_degree_supplier_score['ewfdsc_cc'] = supplier.groupby(['firm_gvkey', 'year', 'quarter'])['supplier_cc'].mean()
equal_weighted_first_degree_supplier_score = equal_weighted_first_degree_supplier_score.reset_index()
customer = customer.dropna()
equal_weighted_first_degree_customer_score = pd.DataFrame()
equal_weighted_first_degree_customer_score['ewfdcc_pol'] = customer.groupby(['firm_gvkey', 'year', 'quarter'])['customer_pol'].mean()
equal_weighted_first_degree_customer_score['ewfdcc_covid'] = customer.groupby(['firm_gvkey', 'year', 'quarter'])['customer_covid'].mean()
equal_weighted_first_degree_customer_score['ewfdcc_cc'] = customer.groupby(['firm_gvkey', 'year', 'quarter'])['customer_cc'].mean()
equal_weighted_first_degree_customer_score = equal_weighted_first_degree_customer_score.reset_index()

In [28]:
equal_weighted_second_degree_supplier_score = supplier[['firm_gvkey', 'supplier_gvkey', 'year', 'quarter']]
equal_weighted_second_degree_supplier_score = equal_weighted_second_degree_supplier_score.merge(equal_weighted_first_degree_supplier_score,
                                                  left_on=['supplier_gvkey', 'year', 'quarter'],
                                                  right_on=['firm_gvkey', 'year', 'quarter'],
                                                  how='left')
equal_weighted_second_degree_supplier_score.drop(['firm_gvkey_y', 'supplier_gvkey'], axis=1, inplace=True)
equal_weighted_second_degree_supplier_score.dropna(inplace=True)
equal_weighted_second_degree_supplier_score.rename({'firm_gvkey_x': 'firm_gvkey',
                                                    'ewfdsc_pol': 'ewsdsc_pol',
                                                    'ewfdsc_covid': 'ewsdsc_covid',
                                                    'ewfdsc_cc': 'ewsdsc_cc'},
                                                   axis='columns',
                                                   inplace=True)
equal_weighted_second_degree_customer_score = customer[['firm_gvkey', 'customer_gvkey', 'year', 'quarter']]
equal_weighted_second_degree_customer_score = equal_weighted_second_degree_customer_score.merge(equal_weighted_first_degree_customer_score,
                                                                                                left_on=['customer_gvkey', 'year', 'quarter'],
                                                                                                right_on=['firm_gvkey', 'year', 'quarter'],
                                                                                                how='left')
equal_weighted_second_degree_customer_score.drop(['firm_gvkey_y', 'customer_gvkey'], axis=1, inplace=True)
equal_weighted_second_degree_customer_score.dropna(inplace=True)
equal_weighted_second_degree_customer_score.rename({'firm_gvkey_x': 'firm_gvkey',
                                                    'ewfdcc_pol': 'ewsdcc_pol',
                                                    'ewfdcc_covid': 'ewsdcc_covid',
                                                    'ewfdcc_cc': 'ewsdcc_cc'},
                                                   axis='columns',
                                                   inplace=True)

In [45]:
value_weighted_first_degree_supplier_score = pd.DataFrame()
value_weighted_first_degree_supplier_score['vwfdsc_pol'] = supplier.groupby(['firm_gvkey', 'year', 'quarter'])['supplier_pol_weighted'].sum()
value_weighted_first_degree_supplier_score['vwfdsc_covid'] = supplier.groupby(['firm_gvkey', 'year', 'quarter'])['supplier_covid_weighted'].sum()
value_weighted_first_degree_supplier_score['vwfdsc_cc'] = supplier.groupby(['firm_gvkey', 'year', 'quarter'])['supplier_cc_weighted'].sum()
value_weighted_first_degree_supplier_score = value_weighted_first_degree_supplier_score.reset_index()
value_weighted_first_degree_customer_score = pd.DataFrame()
value_weighted_first_degree_customer_score['vwfdcc_pol'] = customer.groupby(['firm_gvkey', 'year', 'quarter'])['customer_pol_weighted'].sum()
value_weighted_first_degree_customer_score['vwfdcc_covid'] = customer.groupby(['firm_gvkey', 'year', 'quarter'])['customer_covid_weighted'].sum()
value_weighted_first_degree_customer_score['vwfdcc_cc'] = customer.groupby(['firm_gvkey', 'year', 'quarter'])['customer_cc_weighted'].sum()
value_weighted_first_degree_customer_score = value_weighted_first_degree_customer_score.reset_index()

In [47]:
value_weighted_second_degree_supplier_score = supplier[['firm_gvkey', 'supplier_gvkey', 'year', 'quarter']]
value_weighted_second_degree_supplier_score = value_weighted_second_degree_supplier_score.merge(value_weighted_first_degree_supplier_score,
                                                                                                left_on=['supplier_gvkey', 'year', 'quarter'],
                                                                                                right_on=['firm_gvkey', 'year', 'quarter'],
                                                                                                how='left')
value_weighted_second_degree_supplier_score.drop(['firm_gvkey_y', 'supplier_gvkey'], axis=1, inplace=True)
value_weighted_second_degree_supplier_score.dropna(inplace=True)
value_weighted_second_degree_supplier_score.rename({'firm_gvkey_x': 'firm_gvkey',
                                                    'vwfdsc_pol': 'vwsdsc_pol',
                                                    'vwfdsc_covid': 'vwsdsc_covid',
                                                    'vwfdsc_cc': 'vwsdsc_cc'},
                                                   axis='columns',
                                                   inplace=True)
value_weighted_second_degree_customer_score = customer[['firm_gvkey', 'customer_gvkey', 'year', 'quarter']]
value_weighted_second_degree_customer_score = value_weighted_second_degree_customer_score.merge(value_weighted_first_degree_customer_score,
                                                                                                left_on=['customer_gvkey', 'year', 'quarter'],
                                                                                                right_on=['firm_gvkey', 'year', 'quarter'],
                                                                                                how='left')
value_weighted_second_degree_customer_score.drop(['firm_gvkey_y', 'customer_gvkey'], axis=1, inplace=True)
value_weighted_second_degree_customer_score.dropna(inplace=True)
value_weighted_second_degree_customer_score.rename({'firm_gvkey_x': 'firm_gvkey',
                                                    'vwfdcc_pol': 'vwsdcc_pol',
                                                    'vwfdcc_covid': 'vwsdcc_covid',
                                                    'vwfdcc_cc': 'vwsdcc_cc'},
                                                   axis='columns',
                                                   inplace=True)

In [48]:
firm = pd.concat([supplier[['firm_gvkey', 'year', 'quarter', 'firm_pol', 'firm_covid', 'firm_cc']], customer[['firm_gvkey', 'year', 'quarter', 'firm_pol', 'firm_covid', 'firm_cc']]]).drop_duplicates()
firm = firm.merge(equal_weighted_first_degree_supplier_score,
                  on=['firm_gvkey', 'year', 'quarter'],
                  how='left')
firm = firm.drop_duplicates()
firm = firm.merge(equal_weighted_first_degree_customer_score,
                  on=['firm_gvkey', 'year', 'quarter'],
                  how='left')
firm = firm.drop_duplicates()
firm = firm.merge(equal_weighted_second_degree_supplier_score,
                  on=['firm_gvkey', 'year', 'quarter'],
                  how='left')
firm = firm.drop_duplicates()
firm = firm.merge(equal_weighted_second_degree_customer_score,
                  on=['firm_gvkey', 'year', 'quarter'],
                  how='left')
firm = firm.drop_duplicates()
firm = firm.merge(value_weighted_first_degree_supplier_score,
                  on=['firm_gvkey', 'year', 'quarter'],
                  how='left')
firm = firm.drop_duplicates()
firm = firm.merge(value_weighted_first_degree_customer_score,
                  on=['firm_gvkey', 'year', 'quarter'],
                  how='left')
firm = firm.drop_duplicates()
firm = firm.merge(value_weighted_second_degree_supplier_score,
                  on=['firm_gvkey', 'year', 'quarter'],
                  how='left')
firm = firm.drop_duplicates()
firm = firm.merge(value_weighted_second_degree_customer_score,
                  on=['firm_gvkey', 'year', 'quarter'],
                  how='left')
firm = firm.drop_duplicates()

In [49]:
firm.to_csv('./firm.csv')

In [50]:
firm.dropna()

,firm_gvkey,year,quarter,firm_pol,firm_covid,firm_cc,ewfdsc_pol,ewfdsc_covid,ewfdsc_cc,ewfdcc_pol,...,vwfdsc_cc,vwfdcc_pol,vwfdcc_covid,vwfdcc_cc,vwsdsc_pol,vwsdsc_covid,vwsdsc_cc,vwsdcc_pol,vwsdcc_covid,vwsdcc_cc
319,31673.0,2006,1,87.762067,0.000000,0.000000,82.766571,0.000000,0.000122,23.546032,...,0.000018,23.546032,0.000000,0.000000,88.224179,0.000000,0.000138,122.934025,0.000000,0.000000
320,31673.0,2006,1,87.762067,0.000000,0.000000,82.766571,0.000000,0.000122,23.546032,...,0.000018,23.546032,0.000000,0.000000,116.171976,0.000000,0.000000,122.934025,0.000000,0.000000
321,31673.0,2006,1,87.762067,0.000000,0.000000,82.766571,0.000000,0.000122,23.546032,...,0.000018,23.546032,0.000000,0.000000,60.775156,0.000000,0.000011,122.934025,0.000000,0.000000
322,31673.0,2006,1,87.762067,0.000000,0.000000,82.766571,0.000000,0.000122,23.546032,...,0.000018,23.546032,0.000000,0.000000,41.453537,0.000000,0.000083,122.934025,0.000000,0.000000
323,31673.0,2006,1,87.762067,0.000000,0.000000,82.766571,0.000000,0.000122,23.546032,...,0.000018,23.546032,0.000000,0.000000,57.094605,0.000000,0.000000,122.934025,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32265,23812.0,2021,4,88.251517,2.325866,0.000276,57.174054,0.000000,0.000000,39.413832,...,0.000000,8.993765,1.640187,0.001774,23.551452,0.344081,0.000503,13.484216,3.161086,0.000000
32422,24344.0,2021,1,46.019328,0.755668,0.000684,152.916333,4.069410,0.000424,74.264352,...,0.000012,2.827662,0.228850,0.000024,115.148414,1.940671,0.000000,12.068862,1.774398,0.000166
32423,24344.0,2021,2,36.630037,0.000000,0.001486,49.546999,3.195935,0.000846,95.047999,...,0.000023,2.398985,0.174226,0.000018,95.158796,0.127129,0.000335,38.729513,1.082551,0.000000
32424,24344.0,2021,3,68.524440,0.000000,0.000333,76.148574,3.518339,0.001359,27.852440,...,0.000037,0.792874,0.126336,0.000042,61.387354,0.917672,0.000367,24.170077,1.625366,0.000319


In [2]:
firm = pd.read_csv('./firm.csv')

In [8]:
firm.loc[(firm['vwfdsc_pol'].notna() | firm['vwfdcc_pol'].notna()) & (firm['vwsdcc_pol'].notna() | firm['vwsdsc_pol'].notna()), 'firm_gvkey'].nunique()

538